## 1. 사용할 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import collections
import pyLDAvis.gensim_models
import gensim.corpora as corpora
import gensim
from gensim import models
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import matplotlib.pyplot as plt

/home/jongwook95.lee/miniforge3/envs/bert_eng/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jongwook95.lee/miniforge3/envs/bert_eng/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


## 2. 데이터 불러오기

### (1) 파일 불러오기
- data_dir: 파일 경로